In [ ]:
# use feature importance for feature selection
from numpy import loadtxt
from numpy import sort
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
# load data

In [ ]:
#dataset = loadtxt('pima-indians-diabetes.csv', delimiter=",")
dataset=pd.read_csv('ytr1.csv',encoding='gbk')
feature1=dataset[np.r_[np.array(dataset.columns[1:10].values)]]######数据用feature，其中[]内为需要参与计算的特征数，例：想看第一到第十个因子对因变量的影响，则填[0:9]
# split data into X and y
X = feature1#特征因子
Y = dataset['需要预测的值标签']
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)
# fit model on all training data

In [ ]:

model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)



In [ ]:
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = XGBClassifier()
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

In [ ]:
# plot feature importance using built-in function
from xgboost import plot_importance
from pylab import mpl
import matplotlib.pyplot as plt
import matplotlib
mpl.rcParams['font.sans-serif'] = ['SimHei']
#matplotlib.rcParams['figure.figsize'] = (8.0, 4.0)
#plt.title("年数据-XGBoost-模型中排名前24的特征及其系数",fontsize=18)
#plt.savefig('LR特征筛选',dpi=300, bbox_inches = 'tight')
#plt.show()
plot_importance(model,max_num_features=24,title='Xgboost特征重要性排序')
fig=plt.gcf()
#fig.set_size_inches(150, 100)
#plt.show()
fig.savefig('XBoost特征.png',dpi=300, bbox_inches = 'tight')
#pyplot.show()

In [ ]:
tezheng=pd.DataFrame(model.feature_importances_,dataset.columns.values[1:10])#其中[]内为原文件所有参与的特征,同上
tezheng=tezheng.sort_values(by=0,axis=0,ascending=False)
tezheng.to_csv('data_tezheng.csv',encoding='gbk')##将特征导出为csv文件，并将重要性排序

<font color=Blue size=4>&emsp;相关系数热力图 <br/><font/>

In [ ]:
###########相关系数热力图
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
#data=pd.read_csv('ytr1.csv',encoding='gbk')###数据用
data=pd.read_csv('34jidu_zenzhang_suoying.csv',encoding='gbk')###数据用
#data1=np.loadtxt(open('feature.csv',dtype='str',skiprows=1)
#data1=pd.read_csv('feature.csv',encoding='gbk')
key=tezheng.index.values[0:18]
#print(data[data1.LABEL])
#data=data[data1.LABEL[0:16]]
data=data[key]
#data=data.iloc[0:7,3:30]
print(np.size(data.corr(),1))
plt.subplots(figsize=(np.size(data.corr(),1),np.size(data.corr(),1)))
sns.heatmap(data.corr(),annot=True,vmax=1,square=True,cmap='Blues')
plt.savefig('')
#plt.title('年数据特征的相关系数热力图',fontsize=28)
plt.title('XGBoost模型-特征的相关系数热力图',fontsize=28)
plt.tick_params(labelsize=24)
#plt.savefig('年数据-LR模型-相关系数热力图',dpi=300)
plt.savefig('XGBoost模型-相关系数热力图',dpi=300)
plt.show()
ax=plt.subplots(figsize=(np.size(data.corr(),1),np.size(data.corr(),1)))
#ax.set_ylabel('')
#ax.set_xlabel('')

<font color=Blue size=4>&emsp;下列为ROC图的绘制 <br/><font/>

In [ ]:
##########此处为ROC图绘制，并可得到AUC值
from sklearn.metrics  import roc_curve,auc
from matplotlib import pyplot as plt
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']
#y_score=LR(threshold=0.009,C=2).fit(X_train,y_train).predict_proba(x_test) #descision_function()不可用
#print('trainscore:'+str(model_lasso.score(X_train,y_train)))
model = XGBClassifier()
#model.fit(X_train, y_train)
y_score=model.fit(X_train, y_train).predict_proba(X_test) #descision_function()不可用
# make predictions for test data and evaluate
y_pred = model.predict(X_test)
#print('testscore:'+str(model_lasso.score(x_test,y_test)))
#print(type(y_score))
fpr,tpr,threshold=roc_curve(y_test,y_score[:, 1])
roc_auc=auc(fpr,tpr)###AUC值
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr,color='darkorange',
lw=2, label='ROC curve (area = %0.2f)' % roc_auc) ###假正率为横坐标，真正率为纵坐标做曲线
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('日数据-XGBoost模型-ROC曲线',fontsize=24)
plt.legend(loc="lower right")
#plt.savefig('XGBoost-ROC',dpi=300, bbox_inches = 'tight')
plt.show()
print(roc_auc)